In [1]:
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input, GlobalAveragePooling2D
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import Hinge

In [2]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [6]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')  # Use linear activation for Hinge loss
])


In [7]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=Hinge(), metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
87/87 [==============================] - 42s 473ms/step - loss: 0.6003 - accuracy: 0.7361 - val_loss: 0.5698 - val_accuracy: 0.7465
Epoch 2/20
87/87 [==============================] - 43s 491ms/step - loss: 0.5257 - accuracy: 0.7354 - val_loss: 0.5733 - val_accuracy: 0.7326
Epoch 3/20
87/87 [==============================] - 41s 473ms/step - loss: 0.4910 - accuracy: 0.7375 - val_loss: 0.4732 - val_accuracy: 0.7396
Epoch 4/20
87/87 [==============================] - 41s 468ms/step - loss: 0.4612 - accuracy: 0.7354 - val_loss: 0.4089 - val_accuracy: 0.7431
Epoch 5/20
87/87 [==============================] - 42s 482ms/step - loss: 0.4018 - accuracy: 0.7686 - val_loss: 0.3385 - val_accuracy: 0.8056
Epoch 6/20
87/87 [==============================] - 41s 474ms/step - loss: 0.3314 - accuracy: 0.8344 - val_loss: 0.2773 - val_accuracy: 0.8472
Epoch 7/20
87/87 [==============================] - 45s 519ms/step - loss: 0.2756 - accuracy: 0.8684 - val_loss: 0.2239 - val_accuracy: 0.8819

In [9]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

18/18 [==============================] - 1s 70ms/step - loss: 0.1048 - accuracy: 0.9583
Test Loss: 0.10479641705751419, Test Accuracy: 0.9583333134651184


# Pretrenowane sieci

### MobileNetV2

In [3]:
from tensorflow.keras.applications import MobileNetV2

# Wykorzystanie pretrenowanych sieci
### MobileNetV2
img_height = img_width = 128

# Ładowanie MobileNetV2 bez górnej warstwy (top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Zamrożenie wagi przetrenowanego modelu
base_model.trainable = False

# Dodanie warstw na wierzchu
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# To jest tylko przykład, można tutaj dostosować liczbę neuronów i warstw
output = Dense(1, activation='linear')(x)  # Przykładowa warstwa wyjściowa dla klasyfikacji binarnej

# Skompletowanie nowego modelu
model = Model(inputs=base_model.input, outputs=output)

# Kompilacja modelu z niestandardową funkcją straty i optymalizatorem
model.compile(optimizer=Adam(lr=0.0001), loss=Hinge(), metrics=['accuracy'])
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20


/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


1399/1399 [==============================] - 21s 14ms/step - loss: 0.1892 - accuracy: 0.9221 - val_loss: 0.0283 - val_accuracy: 0.9826
Epoch 2/20
1399/1399 [==============================] - 22s 16ms/step - loss: 0.0662 - accuracy: 0.9728 - val_loss: 0.0496 - val_accuracy: 0.9653
Epoch 3/20
1399/1399 [==============================] - 25s 18ms/step - loss: 0.0596 - accuracy: 0.9764 - val_loss: 0.0442 - val_accuracy: 0.9861
Epoch 4/20
1399/1399 [==============================] - 25s 18ms/step - loss: 0.0492 - accuracy: 0.9828 - val_loss: 0.1670 - val_accuracy: 0.9236
Epoch 5/20
1399/1399 [==============================] - 22s 15ms/step - loss: 0.0525 - accuracy: 0.9821 - val_loss: 0.0394 - val_accuracy: 0.9896
Epoch 6/20
1399/1399 [==============================] - 19s 14ms/step - loss: 0.0276 - accuracy: 0.9871 - val_loss: 0.0313 - val_accuracy: 0.9896
Epoch 7/20
1399/1399 [==============================] - 15s 10ms/step - loss: 0.0324 - accuracy: 0.9857 - val_loss: 0.0219 - val_accura

In [4]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 98ms/step - loss: 0.0413 - accuracy: 0.9792
Test Loss: 0.041338786482810974, Test Accuracy: 0.9791666865348816
